# WINDOW FUNCTION GSL PERIODOGRAMS - TESS LC BATCH FILE PROCESSING

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook calculates the periodograms for the Window Functions of the TESS LC curves. It saves a simple text file with the periodogram result and the figure of the `gls` object. Notice that, in this case, the error for the window function "radial velocities" values is set to a constant value of 1 (so as the weighting of the periodogram has no impact), and the "constant" flux is set 1o 1000. The reason is that trying to set errors to 0.001 electrons / s yielded ann error when trying to overwrite the `flux_err` column of `lightkurve` object.

Error control is implemented to prevent exceptions and timeouts to interrupt the processing. In case any of these errors happen, it is so recorded in the results table. Timeout value is dinamically updated, as the calculation for each record is completed.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $5 min$ and $10 h$. In days, this corresponds to a range of $P\in[0.003472, 0.416667] d$ or, equivalently, frequencies in the range $f\in[288.0, 2.4] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [1]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

import warnings

#from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [8]:
# Configuration:
# Files and folders:
GTO_FILE = "../data/CARM_VIS_objects_with_PG_and_WF.csv"
IN_LC_FOLDER = "../data/CARMENES_GTO_TESS_lc/lightcurves/"
OUT_IMG_FOLDER = "../data/CARM_VIS_TESS_WinFunc_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARM_VIS_TESS_WinFunc_PGs/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
# Periodogram constants:
FBEG = 2.4 # d^{-1}, corresponds to a period P=10 hours
FEND = 288 # d^{-1}, corresponds to a period P=5 min
NUM_POINTS = 285601 # 285600 for TESS LC time-series
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
#FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
FREQ = np.linspace(start=FBEG, stop=FEND, num=NUM_POINTS)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [9]:
def tess_lc_load(filename: str):
    '''Load the TESS LC file and returns a lightcurve object with just the three relevant columns
    Note: it seems to be necessary to drop 'nan' values for GLS to work properly.'''
    lc_lk = lk.read(filename).remove_nans()
    return lc_lk

## Data processing

### GTO data loading

In [10]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS,PG_file_RV,PG_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23585+076_RV_PG.dat,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23556-061_RV_PG.dat,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23548+385_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23548+385_RV_PG.dat
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23505-095_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23505-095_RV_PG.dat
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23492+024_RV_PG.dat,NaN


In [11]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

In [12]:
gto.loc[72, ['Karmn', 'has_lc', 'lc_file']]

Karmn                                             J19025+754
has_lc                                                  True
lc_file    ../data/CARMENES_GTO_TESS_lc/lightcurves/J1902...
Name: 72, dtype: object

Generate the proper auxiliary columns (with the basic periodograms results).

In [ ]:
# Additional columns:
if 'WF_maxP_PG_TESS' in gto.columns:
    pass
else:
    gto['WF_maxP_PG_TESS'] = None # Max power value in the PG
    gto['WF_maxSNR_PG_TESS'] = None # Max power value in the PG
    gto['WF_rms_PG_TESS'] = None # RMS value in the PG residuals
    gto['WF_f_PG_TESS'] = None # Best frequency in the PG (d^(-1))
    gto['WF_e_f_PG_TESS'] = None # Error of the best frequency in the PG (d^(-1))
    gto['WF_Pd_PG_TESS'] = None # Best period in the PG (d)
    gto['WF_e_Pd_PG_TESS'] = None # Error of the best period in the PG (d)
    gto['WF_Ph_PG_TESS'] = None # Best period in the PG (hours)
    gto['WF_e_Ph_PG_TESS'] = None # Error of the best period in the PG (hours)
    gto['WF_Pm_PG_TESS'] = None # Best period in the PG (minutes)
    gto['WF_e_Pm_PG_TESS'] = None # Error of the best period in the PG (minutes)
    gto['WF_A_PG_TESS'] = None # Amplitude of the best frequency
    gto['WF_e_A_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['WF_ph_PG_TESS'] = None # Amplitude of the best frequency
    gto['WF_e_ph_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['WF_T0_PG_TESS'] = None # Reference epoch of the best frequency
    gto['WF_e_T0_PG_TESS'] = None # Error of the epoch of the best frequency
    gto['WF_offset_PG_TESS'] = None # Offset of the best frequency
    gto['WF_e_offset_PG_TESS'] = None # Error of the offset of the best frequency
    gto['WF_FAP_PG_TESS'] = None # False alarm probability
    gto['WF_valid_PG_TESS'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['WF_error_PG_TESS'] = None # The error raised during processing. Empty if processing was successful.
    gto['WF_elapsed_time_PG_TESS'] = None # The time elapsed in calculation
    gto['WF_plain_file_TESS'] = None # The name of the processed file.
    gto['WF_fig_file_TESS'] = None # The name of the figure file.


In [7]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Batch processing of all TESS files

In [10]:
n = len(gto)
n

390

#### Batch processing

In [11]:
# Prevent warnings to produce too much output
warnings.filterwarnings('ignore')

# Batch processing:
lapse_list = []
median_lapse = None
for i in range(0, 3): # FOR TESTING
#for i in [2]: # To repeat the calculation for specific objects.
#for i in range(0, 100):
#for i in range(100, 200):
#for i in range(200, 207):
#for i in range(207, 245):
#for i in range(245, 300):
#for i in range(300, 311):
#for i in range(311, len(gto)):
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # KARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, window function PG calculation started  at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star (TIC: %s)..." %(karmn, tic_id))
    #if True:
    try:
        # LOAD LC FILE:
        lc_file = gto.loc[i, 'lc_file']
        print("filename: %s" %lc_file)
        lc_lk = tess_lc_load(lc_file)

        # GENERATE THE WINDOW FUNCTION
        # All values to 1.0 and errors to 0.0,time points are the same.
        lc_lk['flux'].fill(1000 * u.electron / u.s)
        lc_lk['flux_err'].fill(1 * u.electron / u.s)

        # GENERATE PERIODOGRAM:
        gls = Gls((lc_lk['time'].value, lc_lk['flux'].value, lc_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)
        
        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'WF_maxP_PG_TESS'] = gls.power.max()
        gto.loc[i, 'WF_maxSNR_PG_TESS'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'WF_rms_PG_TESS'] = gls.rms
        gto.loc[i, 'WF_f_PG_TESS'] = gls.best['f']
        gto.loc[i, 'WF_e_f_PG_TESS'] = gls.best['e_f']
        gto.loc[i, 'WF_Pd_PG_TESS'] = gls.best['P']
        gto.loc[i, 'WF_e_Pd_PG_TESS'] = gls.best['e_P']
        gto.loc[i, 'WF_Ph_PG_TESS'] = 24.0 * gls.best['P']
        gto.loc[i, 'WF_e_Ph_PG_TESS'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'WF_Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'WF_e_Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'WF_A_PG_TESS'] = gls.best['amp']
        gto.loc[i, 'WF_e_A_PG_TESS'] = gls.best['e_amp']
        gto.loc[i, 'WF_ph_PG_TESS'] = gls.best['ph']
        gto.loc[i, 'WF_e_ph_PG_TESS'] = gls.best['e_ph']
        gto.loc[i, 'WF_T0_PG_TESS'] = gls.best['T0']
        gto.loc[i, 'WF_e_T0_PG_TESS'] = gls.best['e_T0']
        gto.loc[i, 'WF_offset_PG_TESS'] = gls.best['offset']
        gto.loc[i, 'WF_e_offset_PG_TESS'] = gls.best['e_offset']
        gto.loc[i, 'WF_FAP_PG_TESS'] = gls.FAP(Pn=None)

        # SAVE THE DATA:
        # (No'fits' file here, we just use the 'toFile' method of 'gls' instance)
        plain_filename = OUT_PROCESSED_FOLDER + "WF_" + karmn + "_TESS_PG.dat"
        gls.toFile(plain_filename, header=False)

        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        figure_title = "Window Function PG for TESS LC: %s (TIC %s), P=%.4f [min], FAP=%.4f%%" \
            %(karmn, tic_id, gto.loc[i, 'WF_Pm_PG_TESS'], 100.0 * gto.loc[i, 'WF_FAP_PG_TESS'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + "WF_" + karmn + "_TESS_PG.jpg"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'WF_valid_PG_TESS'] = 1
        gto.loc[i, 'WF_error_PG_TESS'] = ""
        gto.loc[i, 'WF_plain_file_TESS'] = plain_filename
        gto.loc[i, 'WF_fig_file_TESS'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'WF_elapsed_time_PG_TESS'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
    #else:
    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "--- Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'WF_valid_PG_TESS'] = 0
        gto.loc[i, 'WF_error_PG_TESS'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'WF_elapsed_time_PG_TESS'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "--- Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'WF_error_PG_TESS'] = gto.loc[i, 'WF_error_PG_TESS'] + "/" + str(e2)



Record: 311, window function PG calculation started  at 09/08/2022, 11:27:03
Previous median lapse time: None
Processing J05084-210 star (TIC: 146539195)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J05084-210/J05084-210_TESS_TIC146539195_S05.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J05084-210_TESS_PG.dat
Elapsed time: 114.99 seconds
... SUCCESS.
Record: 312, window function PG calculation started  at 09/08/2022, 11:28:58
Previous median lapse time: 114.99 seconds
Processing J05062+046 star (TIC: 455029978)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J05062+046/J05062+046_TESS_TIC455029978_S05.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J05062+046_TESS_PG.dat
Elapsed time: 122.00 seconds
... SUCCESS.
Record: 313, window function PG calculation started  at 09/08/2022, 11:31:00
Previous median lapse time: 118.49 seconds
Processing J05033-173 star (TIC: 169304560)...
filename: ../data/CARMENES_GTO_TE

Elapsed time: 149.01 seconds
... SUCCESS.
Record: 331, window function PG calculation started  at 09/08/2022, 11:58:46
Previous median lapse time: 125.14 seconds
Processing J04173+088 star (TIC: 283866910)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04173+088/J04173+088_TESS_TIC283866910_S05.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J04173+088_TESS_PG.dat
Elapsed time: 159.48 seconds
... SUCCESS.
Record: 332, window function PG calculation started  at 09/08/2022, 12:01:25
Previous median lapse time: 125.43 seconds
Processing J04167-120 star (TIC: 70899085)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04167-120/J04167-120_TESS_TIC70899085_S05.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J04167-120_TESS_PG.dat
Elapsed time: 148.65 seconds
... SUCCESS.
Record: 333, window function PG calculation started  at 09/08/2022, 12:03:54
Previous median lapse time: 125.77 seconds
Processing J04153-076 star (TI

Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J02489-145W_TESS_PG.dat
Elapsed time: 156.11 seconds
... SUCCESS.
Record: 351, window function PG calculation started  at 09/08/2022, 12:32:44
Previous median lapse time: 134.08 seconds
Processing J02489-145E star (TIC: 29960109)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J02489-145E/J02489-145E_TESS_TIC29960109_S04.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J02489-145E_TESS_PG.dat
Elapsed time: 152.03 seconds
... SUCCESS.
Record: 352, window function PG calculation started  at 09/08/2022, 12:35:16
Previous median lapse time: 134.27 seconds
Processing J02486+621 star (TIC: 51024887)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J02486+621/J02486+621_TESS_TIC51024887_S18.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J02486+621_TESS_PG.dat
Elapsed time: 146.07 seconds
... SUCCESS.
Record: 353, window function PG calculation started

Elapsed time: 164.45 seconds
... SUCCESS.
Record: 368, window function PG calculation started  at 09/08/2022, 12:59:55
Previous median lapse time: 134.43 seconds
Processing J01433+043 star (TIC: 369534824)...
filename: nan
--- Some ERROR happened with record #368, J01433+043 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 369, window function PG calculation started  at 09/08/2022, 12:59:55
Previous median lapse time: 134.35 seconds
Processing J01352-072 star (TIC: 29853348)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01352-072/J01352-072_TESS_TIC29853348_S03.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J01352-072_TESS_PG.dat


Elapsed time: 156.62 seconds
... SUCCESS.
Record: 370, window function PG calculation started  at 09/08/2022, 13:02:32
Previous median lapse time: 134.43 seconds
Processing J01339-176 star (TIC: 54003038)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01339-176/J01339-176_TESS_TIC54003038_S03.fits
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J01339-176_TESS_PG.dat


Elapsed time: 159.32 seconds
... SUCCESS.
Record: 371, window function PG calculation started  at 09/08/2022, 13:05:11
Previous median lapse time: 135.23 seconds
Processing J01125-169 star (TIC: 610210976)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01125-169/J01125-169_TESS_TIC439403362_S03.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J01125-169_TESS_PG.dat
Elapsed time: 164.32 seconds
... SUCCESS.
Record: 372, window function PG calculation started  at 09/08/2022, 13:07:56
Previous median lapse time: 136.03 seconds
Processing J01066+192 star (TIC: 243185500)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01066+192/J01066+192_TESS_TIC243185500_S17.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J01066+192_TESS_PG.dat


Elapsed time: 162.51 seconds
... SUCCESS.
Record: 373, window function PG calculation started  at 09/08/2022, 13:10:38
Previous median lapse time: 138.12 seconds
Processing J01056+284 star (TIC: 15615674)...
filename: nan
--- Some ERROR happened with record #373, J01056+284 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 374, window function PG calculation started  at 09/08/2022, 13:10:38
Previous median lapse time: 136.03 seconds
Processing J01048-181 star (TIC: 268633473)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01048-181/J01048-181_TESS_TIC268633473_S03.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J01048-181_TESS_PG.dat
Elapsed time: 152.46 seconds
... SUCCESS.
Record: 375, window function PG calculation started  at 09/08/2022, 13:13:11
Previous median lapse time: 138.12 seconds
Processin

Elapsed time: 191.58 seconds
... SUCCESS.
Record: 383, window function PG calculation started  at 09/08/2022, 13:38:01
Previous median lapse time: 144.49 seconds
Processing J00286-066 star (TIC: 40082634)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J00286-066/J00286-066_TESS_TIC40082634_S03.fits
Results have been written to file:  ../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00286-066_TESS_PG.dat
Elapsed time: 175.05 seconds
... SUCCESS.
Record: 384, window function PG calculation started  at 09/08/2022, 13:40:56
Previous median lapse time: 145.26 seconds
Processing J00184+440 star (TIC: 440109718)...
filename: nan
--- Some ERROR happened with record #384, J00184+440 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 385, window function PG calculation started  at 09/08/2022, 13:40:56
Previous median lapse time: 144.49 seconds
Processing

In [12]:
gto.loc[gto['lc_file'] == '<ambiguous>', ['Karmn']]

,Karmn


## Check the calculations that took longer to complete

We now check the calcultions that took longer, so as to try to repeat them (as it could be due to a problem with the computer itself - for example, the program stopped when the computer went into "sleep" mode).

In [13]:
gto[['Karmn', 'elapsed_time_PG_TESS']].sort_values(by='elapsed_time_PG_TESS', ascending=False) \
    .head(10)

,Karmn,elapsed_time_PG_TESS
97,J18022+642,473.270329
297,J06011+595,392.561480
296,J06024+498,381.279508
295,J06103+821,378.853904
292,J06318+414,363.278000
298,J06000+027,359.406616
288,J06548+332,344.975895
289,J06421+035,344.335647
285,J07001-190,336.867363
284,J07033+346,335.979931


## Summary of calculated periodograms

Number of objects with a TESS file available:

In [14]:
gto[~gto['lc_file'].isna()]['lc_file']

2      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2354...
3      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2350...
5      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2343...
7      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2338...
10     ../data/CARMENES_GTO_TESS_lc/lightcurves/J2324...
                             ...                        
381    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0040...
382    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0038...
383    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0028...
385    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0018...
389    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0005...
Name: lc_file, Length: 271, dtype: object

There are 271 objects in the initial sample that do have a TESS light curve file available.

Number of objects with Window Function TESS PG properly calculated:

In [15]:
gto[gto['WF_valid_PG_TESS'] == 1.0]

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_TESS,WF_e_T0_PG_TESS,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.689208,0.027575,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1353.926624,0.036481,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
5,J23431+365,GJ 1289,-,1289,355.781509,36.536310,23:43:06.31,+36:32:13.1,107.922839,-24.336479,...,1764.702841,0.028355,999.999512,4.306074e-06,1.0,1.0,NaN,97.939914,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23431+36...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
7,J23381-162,G 273-093,-,4352,354.532687,-16.236514,23:38:08.16,-16:14:10.2,61.845437,-69.825220,...,1353.926631,0.036480,1000.000122,9.022946e-07,1.0,1.0,NaN,136.603404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23381-16...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
10,J23245+578,BD+57 2735,-,895,351.126628,57.853057,23:24:30.51,+57:51:15.5,111.552287,-3.085183,...,1955.851155,0.018437,999.999512,3.720858e-06,1.0,1.0,NaN,131.327304,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23245+57...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,J00403+612,2MASS J00402129+6112490,-,NaN,10.089157,61.213379,00:40:21.29,+61:12:49.1,121.597733,-1.629661,...,1764.687712,NaN,1000.000000,NaN,NaN,1.0,,176.113404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00403+61...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
382,J00389+306,Wolf 1056,-,26,9.754064,30.616338,00:38:59.04,+30:36:58.4,119.774312,-32.179243,...,1764.682476,0.027265,999.999878,1.071631e-06,1.0,1.0,,191.575017,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00389+30...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
383,J00286-066,GJ 1012,-,1012,7.162970,-6.667219,00:28:39.46,-06:39:49.2,107.085752,-68.835040,...,1385.744584,0.030548,1000.000305,2.678387e-06,1.0,1.0,,175.054456,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00286-06...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
385,J00183+440,GX And,A,15A,4.613226,44.024787,00:18:22.88,+44:01:22.6,116.690592,-18.446865,...,1764.644951,0.030860,999.999756,2.134945e-06,1.0,1.0,,191.241441,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00183+44...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...


All of the 271 for wich periodogram was calculated without trouble (as opposed to TESS PG, which yielded just 267):

In [17]:
len(gto[(gto['WF_valid_PG_TESS'] == 1.0) & (~gto['lc_file'].isna())])

271

In [18]:
gto[(gto['WF_valid_PG_TESS'] == 0.0) & (~gto['lc_file'].isna())][['Karmn', 'lc_file', 'WF_error_PG_TESS']]

,Karmn,lc_file,WF_error_PG_TESS


In [19]:
gto[(gto['WF_valid_PG_TESS'] == 0.0) & (~gto['lc_file'].isna())]['lc_file'].values

array([], dtype=object)

In [20]:
gto[(gto['WF_valid_PG_TESS'] == 0.0) & (~gto['lc_file'].isna())]['WF_error_PG_TESS'].values

array([], dtype=object)

Maybe it seems to be something to do with the generation of the fits files.

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the available TESS light curves for GTO objetcs, and stored the results.
- Out of the initial 390 objects:
  - 271 stars do have a TESS light curve available. For all of them it was possible to calculate the periodogram for the Window Function (as opposed to TESS PG, which only worked for 267 objects):
